<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/dataset_class/mmseg4j/mmseg4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53924 sha256=9b33a4c989d794971ec70351689e8ba13f39cbae3fb5f03a7a32538cd9b6a83c
  Stored in directory: /root/.cache/pip/wheels/78/ae/2e/5ce789557ca59281a463dabe9d6522e429be240322148e5379
Successfully built docx


In [2]:
!pip install exceptions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement exceptions (from versions: none)
ERROR: No matching distribution found for exceptions


In [3]:
import pandas as pd
import os
import numpy as np
import re
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
save_file_csv = 'anthelion-master_class_all.csv'
file_csv =  'anthelion-master.csv'
file_dir = '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j'

In [15]:
fileList = []
def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
         for file in files:
            if file[-4:] == '.txt':
                fileList.append(os.path.join(root, file))

In [16]:
file_name(file_dir)

In [17]:
fileList

['/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/MaxWordSeg.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/ComplexSeg.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/Chunk.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/Simple.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/Word.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/MaxMatchRule.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/Sentence.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/SimpleSeg.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/Dictionary.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/CharNode.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/Seg.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/Rule.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/MMSeg.txt']

In [18]:
# stopwords
stopwords = ['abstract', 'java','assert', 'boolean', 'break', 'byte', 'case', 'catch', 'class','char', 'continue', 'default', 'do', 'double', 'else', 'enum', 'extends', 'final', 'finally', 'float', 'for', 'if', 'implements', 'import', 'int', 'interface', 'long', 'native', 'new', 'package','private','public', 'protected',  'return', 'short', 'static', 'strictfp', 'super', 'switch', 'synchronized', 'this', 'throw', 'transient', 'try', 'void', 'volatile', 'while']
dicts = {'\n': '','!': '', '"': '','：':'', '#': '', '$': '', '%': '', '&': '', "'": '', '(': '', ')': '', '*': '', '+': '', ',': '', '-': '', '.': '', '/': '', ':': '', ';': '', '<': '', '=': '', '>': '', '?': '', '@': '', '[': '', '\\': '', ']': '', '^': '', '_': '', '`': '', '{': '', '|': '', '}': '', '~': ''}
# stopwords = {'abstract':'', 'java':'','assert':'', 'boolean':'', 'break':'', 'byte':'', 'case':'', 'catch':'', 'class':'','char':'', 'continue':'', 'default':'', 'do':'', 'double':'', 'else':'', 'enum':'', 'extends':'', 'final':'', 'finally':'', 'float':'', 'for':'', 'if':'', 'implements':'', 'import':'', 'int':'', 'interface':'', 'long':'', 'native':'', 'new':'', 'package':'','private':'','public':'', 'protected':'',  'return':'', 'short':'', 'static':'', 'strictfp':'', 'super':'', 'switch':'', 'synchronized':'', 'this':'', 'throw':'', 'transient':'', 'try':'','void':'', 'volatile':'', 'while':''}

In [19]:
def lm_find_unchinese(line):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    unchinese = re.sub(pattern,"",line)
    unchinese = re.sub('[{}]'.format(punctuation),"",unchinese)
    #print("unchinese:",unchinese)
    return unchinese

In [20]:
def del_stopwords(line):

    punc_table = str.maketrans(dicts)
    new_line = line.translate(punc_table)


    return new_line


In [21]:
index = []
index_all = []
class_name = []
Brain_Class = []
for i, f in enumerate(fileList):
    with open(f,encoding='gb18030',errors='ignore') as file:
        list = []
        list2 = []
        is_brain = 0
        filename = os.path.basename(file.name)
        name = filename[:-4]
        class_name.append(name)
        list.append(name)
        list2.append(name)
        
        for line in file.readlines():
            line = lm_find_unchinese(line)
            line = del_stopwords(line)
            values = line.split(' ')
            coefs = values[1:-1]
            coefs = [i for i in coefs if i != '']
            keyword_brain = 'hongshuai'            

            if  keyword_brain in line:
                is_brain = 1
            if len(coefs) != 0:
                if coefs != ' ':
                    if coefs != '\n':
                        for i in coefs:
                            list.append(i)

        index.append(list)
        
        if is_brain == 0:
            list2.append(False)
        else :
            list2.append(True)
            print('T')
        
#         print (list2)
        
        Brain_Class.append(list2)
            


In [22]:
Brain_Class = np.array(Brain_Class)
Brain_Class

array([['MaxWordSeg', 'False'],
       ['ComplexSeg', 'False'],
       ['Chunk', 'False'],
       ['Simple', 'False'],
       ['Word', 'False'],
       ['MaxMatchRule', 'False'],
       ['Sentence', 'False'],
       ['SimpleSeg', 'False'],
       ['Dictionary', 'False'],
       ['CharNode', 'False'],
       ['Seg', 'False'],
       ['Rule', 'False'],
       ['MMSeg', 'False']], dtype='<U12')

In [23]:
def additem(list):
    if len(list)<3:
        q = len(list)-1
        str = '_1'
        str2 = '_2'
        item = ''.join([list[q], str])
        item_2 = ''.join([list[q], str2])
        # item = list[q]
        list.append(item)
        list.append(item_2)
        # print(list)
    return list

for item in index_all:
    # print(item)
    item = additem(item)
    # print(item)

In [24]:
news_df = np.array(index,dtype=object)
print(news_df)

[list(['MaxWordSeg', 'MaxWordSegDictionary'])
 list(['ComplexSeg', 'ComplexSegDictionary', 'otherRulesaddnew', 'otherRulesaddnew', 'otherRulesaddnew', 'Chunk', 'createChunkSentence', 'senchar', 'chsint', 'tailLenint', 'offsetsCharNode', 'Chunk', 'cknew', 'for', 'int', 'i0', 'i', '3', 'i', 'if', 'offsetsi', 'chslength', 'ckwordsinew', 'WordchssengetStartOffsetoffsetsitailLeni', 'if', 'tailLeni', '0', 'CharNode', 'if', 'cn', 'null', 'return', 'static', 'boolean', 'return', 'static', 'void', 'setShowChunkboolean'])
 list(['Chunk', 'Largest', 'Average', 'Word', 'double', 'if', 'avgLen', '0', 'avgLendoublegetLen', 'return', 'int', 'if', 'count', '0', 'for', 'Word', 'word', 'words', 'if', 'word', 'null', 'return', 'Word', 'int', 'if', 'len', '0', 'for', 'Word', 'word', 'words', 'if', 'word', 'null', 'return', 'Sum', 'of', 'Degree', 'of', 'Morphemic', 'Freedom', 'of', 'int', 'if', 'sumDegree', '0', 'int', 'for', 'Word', 'word', 'words', 'if', 'word', 'null', 'wordgetDegree', '1', 'return', 'V

In [25]:

def f_lsa(news_df, n):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(news_df)
  
    n_cap = 1
    svd_model = TruncatedSVD(n_components=n_cap) 
    X2 = svd_model.fit_transform(X) 
    terms = vectorizer.get_feature_names()
    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:7]
        # print("Topic " + str(i) + ": ")
        for t in sorted_terms:
            print(t[0])
    print(" ")

    D = svd_model.components_
    T = X2
    S = np.diagflat(svd_model.singular_values_)

    X = np.dot(T, S)
    X = np.dot(X, D)
    c = np.dot(X, X.T)

    acsm = None
    sum = 0
    for i in c:
        for j in i:
            if j < 0:
                j = 0
            sum += j
        acsm = sum / n / n

    return acsm

In [26]:
q = 0
acsm= []
for i in index:
    lists = []
    print("*************",i,"****************")
    for j in i:
        lists.append(j)
    if len(lists) > 1:
        acsm.append(f_lsa(lists,len(i)))
    else :
        acsm.append(1)

************* ['MaxWordSeg', 'MaxWordSegDictionary'] ****************
maxwordsegdictionary
maxwordseg
 
************* ['ComplexSeg', 'ComplexSegDictionary', 'otherRulesaddnew', 'otherRulesaddnew', 'otherRulesaddnew', 'Chunk', 'createChunkSentence', 'senchar', 'chsint', 'tailLenint', 'offsetsCharNode', 'Chunk', 'cknew', 'for', 'int', 'i0', 'i', '3', 'i', 'if', 'offsetsi', 'chslength', 'ckwordsinew', 'WordchssengetStartOffsetoffsetsitailLeni', 'if', 'tailLeni', '0', 'CharNode', 'if', 'cn', 'null', 'return', 'static', 'boolean', 'return', 'static', 'void', 'setShowChunkboolean'] ****************
if
i0
complexseg
null
tailleni
boolean
charnode
 
************* ['Chunk', 'Largest', 'Average', 'Word', 'double', 'if', 'avgLen', '0', 'avgLendoublegetLen', 'return', 'int', 'if', 'count', '0', 'for', 'Word', 'word', 'words', 'if', 'word', 'null', 'return', 'Word', 'int', 'if', 'len', '0', 'for', 'Word', 'word', 'words', 'if', 'word', 'null', 'return', 'Sum', 'of', 'Degree', 'of', 'Morphemic', 'Fr

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


string
systemoutprintnmmseg4j
seg
return
boolean
simple
txtstring
 
************* ['Word', 'int', 'return', 'int', 'return', 'getStartOffset', 'int', 'return', 'char', 'return', 'String', 'return', 'new', 'String', 'return', 'char', 'sen', 'see', 'int', 'return', 'void', 'setDegreeint', 'void', 'setTypeString', 'param', 'startOffset', 'Wordchar', 'wordint', 'senoffset', 'param', 'senStartOffset', 'param', 'offset', 'param', 'Wordchar', 'senint', 'senStartOffsetint', 'offsetint', 'param', 'startOffset', 'Wordchar', 'wordint', 'startOffsetString'] ****************
return
setdegreeint
word
getstartoffset
offsetint
void
settypestring
 
************* ['MaxMatchRule', 'void', 'addChunkChunk', 'if', 'chunkgetLen', 'maxLen', 'protected', 'boolean', 'isRemoveChunk', 'return', 'chunkgetLen', 'void'] ****************
void
boolean
addchunkchunk
return
isremovechunk
if
maxlen
 
************* ['Sentence', 'void', 'addOffsetint', 'offset', 'int', 'return', 'int', 'return', 'char', 'return', 'boolean'

In [27]:
c = []
for i in range(len(class_name)):
      for j in range(len(acsm)):
        if i==j:
            t=[class_name[i],acsm[j]]
            c.append(t)
c = np.array(c)


In [28]:
from tensorflow.keras.layers import Input
from keras.models import Model
from keras.layers import Dense

In [29]:
def word2vec_train(index):
    list = []
    list.append(index[0])
    words = []

    additem(index)
    print(index)
    for word in index:
        if word not in words:
            words.append(word)

    word2int = {}
    for i,word in enumerate(words):
        word2int[word] = i
    print(word2int)



    WINDOW_SIZE=2
    data = []
    for idx,word in enumerate(index):
        for neighbor in index[max(idx - WINDOW_SIZE,0) : min(idx + WINDOW_SIZE,len(index))+ 1]:
            if neighbor != word:
                data.append([word, neighbor])
    df = pd.DataFrame(data,columns=['input','label'])


    ONE_HOT_DIM = len(words)
    def to_one_hot_encoding(data_point_index):
        one_hot_encoding = np.zeros(ONE_HOT_DIM)
        one_hot_encoding[data_point_index] = 1
        return one_hot_encoding

    X = []
    Y = []

    for x, y in zip(df['input'],df['label']):
        X.append(to_one_hot_encoding(word2int[x]))
        Y.append(to_one_hot_encoding(word2int[y]))

    x_train = np.array(X)
    y_train = np.array(Y)
    embed_size = 3
    xx = Input(shape=(x_train.shape[1],))
    yy = Dense(units=embed_size, activation='linear')(xx)
    yy =Dense(units=y_train.shape[1],activation='softmax')(yy)
    model = Model(inputs=xx, outputs=yy)
    model.compile(loss = 'categorical_crossentropy',optimizer = 'adam')
    model.fit(x=x_train,y=y_train,
            batch_size=16,
            epochs=10
    )
    weights = model.get_weights()[0]

    for word in words:
        q = weights[df.get(word)]

        arr_new = np.sum(q, axis=1)
        print('-----------------------')
        print(arr_new.shape)
        print('-----------------------')
        arr_new = arr_new.tolist()

        list.append(arr_new)

        break
    return list


In [30]:
embedding = []
for item in index:
    embedding.append(word2vec_train(item))
print(embedding)

['MaxWordSeg', 'MaxWordSegDictionary', 'MaxWordSegDictionary_1', 'MaxWordSegDictionary_2']
{'MaxWordSeg': 0, 'MaxWordSegDictionary': 1, 'MaxWordSegDictionary_1': 2, 'MaxWordSegDictionary_2': 3}
Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 1.4404
Epoch 2/10
1/1 [==============================] - 0s 20ms/step - loss: 1.4389
Epoch 3/10
1/1 [==============================] - 0s 27ms/step - loss: 1.4375
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 1.4360
Epoch 5/10
1/1 [==============================] - 0s 9ms/step - loss: 1.4345
Epoch 6/10
1/1 [==============================] - 0s 10ms/step - loss: 1.4331
Epoch 7/10
1/1 [==============================] - 0s 18ms/step - loss: 1.4316
Epoch 8/10
1/1 [==============================] - 0s 14ms/step - loss: 1.4302
Epoch 9/10
1/1 [==============================] - 0s 13ms/step - loss: 1.4288
Epoch 10/10
1/1 [==============================] - 0s 8ms/step - loss: 1.4273
-----------------------
(1, 3

In [33]:
read_file = pd.read_excel ("/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/mmseg4j.xls")
read_file.to_csv ("/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/mmseg4j.csv", 
                  index = None,
                  header=True)
df =pd.read_csv('/content/drive/MyDrive/delesmell/dataset_class/mmseg4j/mmseg4j.csv') 

In [34]:
x= df.iloc[:, 0]

In [35]:
ngList = []

In [36]:
for index in x:
    q = 1
    for i in range(0,len(embedding)):
        st = str(embedding[i][0])
        st = st[1:]
        if index == st:
            ngList.append(embedding[i][1])
            q = 0
    if q==1 :
        ngList.append([[1, 1, 1]])



word2vec = []
list1 = []
list2 = []
list3 = []
for i in range(0,len(ngList)):
    word2vec.append(ngList[i][0])

for i in range(0,len(word2vec)):
    list1.append(word2vec[i][0])
for i in range(0,len(word2vec)):
    list2.append(word2vec[i][1])
for i in range(0,len(word2vec)):
    list3.append(word2vec[i][2])


In [37]:
for i in range(0,len(Brain_Class)):
    if 'False' == Brain_Class[i][1]:
        q = 1
    else :
        print('*')

In [38]:
len(Brain_Class[0][0])

10

In [39]:

ngList=[]

for index in x:
    q = 1
    for i in range(0,len(c)):
        st = str(c[i][0])
        st = st[1:]
        if index == st:
            ngList.append(c[i][1])
            q = 0
    if q==1 :
        ngList.append(1)
        

In [40]:
brainList=[]

for index in x:
    q = 1
    for i in range(0,len(c)):
        st = str(Brain_Class[i][0])
        st = st[1:]
        if index == st:
            brainList.append(Brain_Class[i][1])
            q = 0
    if q==1 :
        brainList.append('0')
        
        
print(len(brainList))
print(len(x))

18
18


In [41]:
column=df.columns.tolist()

In [42]:
df['acsm'] = ngList
df['word2vec'] = list1
df['word2vec2'] = list2
df['word2vec3'] = list3
df['is_brain'] = brainList
df.to_csv(save_file_csv,mode = 'a',index =False)

In [43]:
df.head()

,NAME,AMW,ATDF,BOVM,BUR,CBO,CC,CM,CRIX,DAC,...,NOM,NOPA,TCC,WMC,WOC,acsm,word2vec,word2vec2,word2vec3,is_brain
0,ComplexSeg,4.40,18,0,0.43,9,0,0,1.0,1,...,5,0,0.5,22,1.0,1,1,1,1,0
1,MaxWordSeg,5.50,6,0,1.00,2,0,0,0.5,0,...,2,0,1.0,11,1.0,1,1,1,1,0
2,SimpleSeg,3.50,5,0,0.14,3,0,0,1.0,0,...,2,0,1.0,7,1.0,1,1,1,1,0
3,MaxMatchRule,1.33,0,0,1.00,1,1,1,0.0,0,...,3,0,1.0,4,1.0,1,1,1,1,0
4,LargestAvgLenRule,1.00,0,0,1.00,1,0,0,0.0,0,...,4,0,1.0,4,1.0,1,1,1,1,0
